In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("Resources/complete_merged_df.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/complete_merged_df.csv'

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

df = df.drop(columns =["Unnamed: 0"], axis=1)
df.head()

In [ ]:
# Determine the number of unique values in each column.
df.nunique()

In [ ]:
# we are going to select top 4 important variables based on the results of RandomForest sampling
nn_df = df[["moid","moid_ld","h","h_cad","pha","v_rel","v_inf"]]
nn_df.head()

In [ ]:
# Determine the number of unique values in each column.
nn_df.nunique()

In [ ]:
# Look at the value counts for binning
moid_counts = nn_df.moid.value_counts()
print(moid_counts)

In [2]:
# Split our preprocessed data into our features and target arrays
y=nn_df.pha
X=nn_df.drop("pha",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

NameError: name 'nn_df' is not defined

In [3]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


NameError: name 'X_train' is not defined

In [11]:
# defining the number of training features
print(len(X_train_scaled[0]))

6


In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 12)                84        
                                                                 
 dense_10 (Dense)            (None, 12)                156       
                                                                 
 dense_11 (Dense)            (None, 1)                 13        
                                                                 
Total params: 253
Trainable params: 253
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [42]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}hdf5"

In [43]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [44]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])


Epoch 1/100
528/528 [==============================] - 1s 2ms/step - loss: 0.2433 - accuracy: 0.9281
Epoch 2/100
528/528 [==============================] - 1s 2ms/step - loss: 0.0562 - accuracy: 0.9826
Epoch 3/100
528/528 [==============================] - 1s 2ms/step - loss: 0.0375 - accuracy: 0.9883
Epoch 4/100
528/528 [==============================] - 1s 2ms/step - loss: 0.0291 - accuracy: 0.9910
Epoch 5/100
507/528 [===========================>..] - ETA: 0s - loss: 0.0250 - accuracy: 0.9922
Epoch 00005: saving model to checkpoints\weights.05hdf5
528/528 [==============================] - 1s 2ms/step - loss: 0.0246 - accuracy: 0.9923
Epoch 6/100
528/528 [==============================] - 1s 2ms/step - loss: 0.0212 - accuracy: 0.9934
Epoch 7/100
528/528 [==============================] - 1s 2ms/step - loss: 0.0193 - accuracy: 0.9941
Epoch 8/100
528/528 [==============================] - 1s 2ms/step - loss: 0.0172 - accuracy: 0.9942
Epoch 9/100
528/528 [==============================

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

176/176 - 0s - loss: 0.0060 - accuracy: 0.9977 - 371ms/epoch - 2ms/step
Loss: 0.005975833162665367, Accuracy: 0.9976884722709656


In [46]:
# Export the model to HDF5 file
nn.save("Resources/NEOs_project_NN.h5")

In [47]:
# Making predictions using the testing data.
predictions = nn.predict(X_test_scaled)
print(len(predictions))

5624


In [48]:
print(predictions)

[[0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 ...
 [0.0000000e+00]
 [1.7818139e-17]
 [0.0000000e+00]]


In [ ]:
df['PHA Predict'] = lambda x: 0 if x < 0.5 else 1
